In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()                     # things we need for Tensorflow
import numpy as np
import tensorflow

#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()

/home/codekhal/miniconda3/envs/av/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/codekhal/miniconda3/envs/av/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/codekhal/miniconda3/envs/av/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/codekhal/miniconda3/envs/av/lib/python3.5/s

In [2]:
# Make Sure that you have installed the downgraded version of tensorflow, since it will not work in tf version 2.0

In [3]:
tensorflow.__version__

'1.14.0'

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd
import pickle
import random

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/codekhal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df = pd.read_json("intents.json")
df

,intents
0,"{'patterns': ['Hi there', 'hey', 'How are you'..."
1,"{'patterns': ['who are you', 'who you', 'who i..."
2,"{'patterns': ['who is chief minister of goa', ..."
3,"{'patterns': ['What is the capital of goa', 'g..."
4,"{'patterns': ['Bye', 'See you later', 'Goodbye..."
5,"{'patterns': ['Thanks', 'Thank you', 'That's h..."
6,"{'patterns': [], 'context': [''], 'tag': 'noan..."
7,"{'patterns': ['Tell me about Goa!', 'what is g..."
8,"{'patterns': ['area of goa', 'geography of goa..."
9,"{'patterns': ['about internet serives', 'how i..."


In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in df['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))# sort classes
classes = sorted(list(set(classes)))# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)
print("Documents\n",documents)

106 documents
26 classes ['About Goa', 'Area geography', 'CM', 'Capital Goa', 'Drinking laws', 'Electricity standards/ plug standards', 'Languages spoken', 'Major religions', 'Places Goa', 'STD Code', 'Safety', 'Things to do in goa', 'Time zones/ Is IST followed', 'Wifi internet', 'bot', 'clothing', 'festivals', 'flights', 'food tips', 'goodbye', 'greeting', 'language', 'medical facilities', 'thanks', 'top hotels', 'weather']
133 unique stemmed words ['!', "'s", ',', 'about', 'airlin', 'am', 'and', 'anyon', 'ar', 'are', 'area', 'awesom', 'broadband', 'bye', 'cal', 'can', 'capit', 'chat', 'chief', 'cloth', 'cm', 'cod', 'codes/', 'condit', 'cult', 'custom', 'day', 'do', 'doe', 'domest', 'drink', 'elect', 'facil', 'fest', 'fet', 'flight', 'follow', 'food', 'for', 'from', 'geograph', 'goa', 'good', 'goodby', 'govern', 'healthc', 'hello', 'help', 'hey', 'hi', 'hospit', 'hotel', 'how', 'humid', 'i', 'in', 'intern', 'internet', 'is', 'ist', 'it', 'langu', 'lat', 'law', 'leis', 'lik', 'liqu', 

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    print(pattern_words,"\n",words)
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print(np.array(train_x).shape,np.array(train_y).shape)
with open(r"/home/codekhal/virtual-tourist-guide/vocab.p","wb") as f:
    pickle.dump((words,classes),f)

['hi', 'ther'] 
 ['!', "'s", ',', 'about', 'airlin', 'am', 'and', 'anyon', 'ar', 'are', 'area', 'awesom', 'broadband', 'bye', 'cal', 'can', 'capit', 'chat', 'chief', 'cloth', 'cm', 'cod', 'codes/', 'condit', 'cult', 'custom', 'day', 'do', 'doe', 'domest', 'drink', 'elect', 'facil', 'fest', 'fet', 'flight', 'follow', 'food', 'for', 'from', 'geograph', 'goa', 'good', 'goodby', 'govern', 'healthc', 'hello', 'help', 'hey', 'hi', 'hospit', 'hotel', 'how', 'humid', 'i', 'in', 'intern', 'internet', 'is', 'ist', 'it', 'langu', 'lat', 'law', 'leis', 'lik', 'liqu', 'maj', 'many', 'me', 'med', 'min', 'minstry', 'nam', 'nat', 'next', 'nic', 'of', 'outfit', 'output', 'plac', 'plug', 'popul', 'rain', 'relax', 'relig', 'resta', 'review', 'roam', 'saf', 'scen', 'see', 'ser', 'should', 'speak', 'spend', 'spok', 'spot', 'standard', 'stay', 'std', 'suit', 'tak', 'tel', 'temp', 'thank', 'that', 'the', 'ther', 'they', 'thi', 'thing', 'til', 'tim', 'tip', 'to', 'top', 'tour', 'tradit', 'typ', 'visit', 'wear

In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [10]:
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("SIHchatbot2020.h5")

Epoch 1/200
106/106 [==============================] - 0s 2ms/sample - loss: 3.2569 - acc: 0.0660
Epoch 2/200
106/106 [==============================] - 0s 400us/sample - loss: 3.2216 - acc: 0.0755
Epoch 3/200
106/106 [==============================] - 0s 396us/sample - loss: 3.1350 - acc: 0.0943
Epoch 4/200
106/106 [==============================] - 0s 396us/sample - loss: 3.1224 - acc: 0.1132
Epoch 5/200
106/106 [==============================] - 0s 613us/sample - loss: 3.0234 - acc: 0.1132
Epoch 6/200
106/106 [==============================] - 0s 403us/sample - loss: 2.9233 - acc: 0.1981
Epoch 7/200
106/106 [==============================] - 0s 413us/sample - loss: 2.7935 - acc: 0.2547
Epoch 8/200
106/106 [==============================] - 0s 379us/sample - loss: 2.6712 - acc: 0.2358
Epoch 9/200
106/106 [==============================] - 0s 407us/sample - loss: 2.6776 - acc: 0.1792
Epoch 10/200
106/106 [==============================] - 0s 446us/sample - loss: 2.4662 - acc: 0.3868
E

106/106 [==============================] - 0s 870us/sample - loss: 0.1739 - acc: 0.9528
Epoch 83/200
106/106 [==============================] - 0s 875us/sample - loss: 0.2124 - acc: 0.9434
Epoch 84/200
106/106 [==============================] - 0s 832us/sample - loss: 0.2496 - acc: 0.9151
Epoch 85/200
106/106 [==============================] - 0s 986us/sample - loss: 0.3039 - acc: 0.9057
Epoch 86/200
106/106 [==============================] - 0s 830us/sample - loss: 0.2038 - acc: 0.9528
Epoch 87/200
106/106 [==============================] - 0s 834us/sample - loss: 0.2295 - acc: 0.9434
Epoch 88/200
106/106 [==============================] - 0s 913us/sample - loss: 0.1732 - acc: 0.9434
Epoch 89/200
106/106 [==============================] - 0s 933us/sample - loss: 0.1437 - acc: 0.9434
Epoch 90/200
106/106 [==============================] - 0s 839us/sample - loss: 0.1921 - acc: 0.9434
Epoch 91/200
106/106 [==============================] - 0s 927us/sample - loss: 0.1376 - acc: 0.9623
Epo

106/106 [==============================] - 0s 821us/sample - loss: 0.0440 - acc: 1.0000
Epoch 163/200
106/106 [==============================] - 0s 763us/sample - loss: 0.1239 - acc: 0.9623
Epoch 164/200
106/106 [==============================] - 0s 768us/sample - loss: 0.1229 - acc: 0.9623
Epoch 165/200
106/106 [==============================] - 0s 723us/sample - loss: 0.1226 - acc: 0.9717
Epoch 166/200
106/106 [==============================] - 0s 762us/sample - loss: 0.1204 - acc: 0.9340
Epoch 167/200
106/106 [==============================] - 0s 773us/sample - loss: 0.1143 - acc: 0.9717
Epoch 168/200
106/106 [==============================] - 0s 823us/sample - loss: 0.1863 - acc: 0.9434
Epoch 169/200
106/106 [==============================] - 0s 808us/sample - loss: 0.0660 - acc: 0.9811
Epoch 170/200
106/106 [==============================] - 0s 888us/sample - loss: 0.0655 - acc: 0.9906
Epoch 171/200
106/106 [==============================] - 0s 835us/sample - loss: 0.1265 - acc: 0

In [11]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    print()
    return(np.array(bag))

In [12]:
p = bow("about goa", words)
print (p)
print (classes)

found in bag: about
found in bag: goa

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['About Goa', 'Area geography', 'CM', 'Capital Goa', 'Drinking laws', 'Electricity standards/ plug standards', 'Languages spoken', 'Major religions', 'Places Goa', 'STD Code', 'Safety', 'Things to do in goa', 'Time zones/ Is IST followed', 'Wifi internet', 'bot', 'clothing', 'festivals', 'flights', 'food tips', 'goodbye', 'greeting', 'language', 'medical facilities', 'thanks', 'top hotels', 'weather']


In [13]:
# Use pickle to load in the pre-trained model
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior() 
'''
global graph
graph = tensorflow.get_default_graph()
with open(r'katana-assistant-model.pkl', 'rb') as f:
    model = pickle.load(f)
'''

"\nglobal graph\ngraph = tensorflow.get_default_graph()\nwith open(r'katana-assistant-model.pkl', 'rb') as f:\n    model = pickle.load(f)\n"

In [14]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [35]:
test = ""

In [36]:
import numpy as np
result=classify_local(test)
#print(result)
for intent in df['intents']:
    if float(result[0][1])>0.6:
        if intent['tag']==result[0][0]:
            print(np.random.choice(intent['responses']))
    else:
        if intent['tag']=='noanswer':
            print(np.random.choice(intent['responses']))

found in bag: chief
found in bag: min
found in bag: of
found in bag: goa

Chief minister of Goa is honorable Mr. Promod Savant, member of the Bhartiya Janta Party. He represents the Sanquelim constituency in goa legislative assembly. Ministry of goa: https://www.goa.gov.in/government/who-is-who/
